In [ ]:
%load_ext sql
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [ ]:
%%sql --save special_campaigns
SELECT 
    campaign.internal_name,
    COUNT(*) count
FROM cts_forms_report report
LEFT JOIN cts_forms_campaign campaign
ON report.origination_utm_campaign_id = campaign.uuid
WHERE report.origination_utm_campaign_id IS NOT NULL
GROUP BY 1
ORDER BY count desc

In [ ]:
%%sql special_campaigns_dated <<
SELECT 
    campaign.internal_name AS name,
    COALESCE(campaign.section, 'No Section') AS section,
    DATE_TRUNC('month', report.create_date) AS month,
    COUNT(1) AS month_total,
    SUM(COUNT(1)) OVER (PARTITION BY campaign.internal_name ORDER BY DATE_TRUNC('month', report.create_date)) AS total
FROM cts_forms_report report
LEFT JOIN cts_forms_campaign campaign
ON report.origination_utm_campaign_id = campaign.uuid
WHERE
    report.origination_utm_campaign_id IS NOT NULL
    AND NOT campaign.archived
GROUP BY campaign.internal_name, campaign.section, DATE_TRUNC('month', report.create_date)
ORDER BY campaign.internal_name, DATE_TRUNC('month', report.create_date)

In [ ]:
import plotly.express as px

fig = px.line(special_campaigns_dated,
              x='month', y='total', color='name',
             hover_data=['name', 'month', 'month_total', 'total', 'section'],
             labels={
                 'month': 'Month',
                 'total': 'Number of Reports Overall',
                 'month_total': 'Added Reports This Month',
                 'name': 'Campaign Name',
                 'section': 'Section',
             }).update_traces(mode="lines+markers")

fig.show()